In [74]:
import numpy as np
import pandas as pd 
import spacy
import string
from nltk.corpus import stopwords

In [75]:
# out-of-the-box: download best-matching default model
#python -m spacy download en
parser = spacy.load("en")

In [3]:
df_train = pd.read_csv('data/train_kaggle.csv', sep=',', encoding='utf-8')

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
id        20800 non-null int64
title     20242 non-null object
author    18843 non-null object
text      20761 non-null object
label     20800 non-null int64
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
df_train['text'].apply(type).unique()
# explore more why the text type is coming as float

print("Before removing float type: ",df_train.shape)
# only remove rows with text as float type

df_train = df_train.drop(df_train[df_train['text'].apply(type) == float].index)
print("After removing float type: ", df_train.shape)

Before removing float type:  (20800, 5)
After removing float type:  (20761, 5)


In [6]:
def clean_text(text):
    # remove new lines 
    text = text.strip().replace("\n", " ")
    text = text.replace("\r", " ")
    
    # remove html tags
    text = text.replace("&amp;", "and")
    text = text.replace("&gt;", ">")
    text = text.replace("&lt;", "<")
    return text



In [7]:
# clean text before passing it to spacy tokenizer
cleaned_text = [ clean_text(text) for text in df_train["text"]]


In [29]:
# stopword that we dont want 
STOPLIST = set(stopwords.words('english')) # add more stopword
# symbols that we dont want, research add more later 
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”"]

# Lets create a custome tokenizer using spacy
def spacy_tokenizer(texts):
    #tokens = parser(sentence)
    # lets replace named entity, if a token is a place replace with <PLACE> if a name with<NAME>
    
    # we need dict to store mapping of each document and their tokens, we also need a hastable or lookup
    # to keep track of their ids
    token_dict = {}
    hash_table = {}
    len_texts = len(texts)
    print(len_texts)
    
    
#     for i, text in enumerate(texts):
#         hash_table[text] = i
    
#     print(hash_table)         
#     for document in parser.pipe(texts, batch_size=1000, n_threads=4):
#         tok_list = []
#         for doc in document:
#             try:
#                 tok_list.append(doc.text().lower().strip() if doc.ent_type_ == "" else "<{}>".format(doc.ent_type_))
#             except:
#                 #print("Erro when identifying NER, setting to UNKOWN type")
#                 tok_list.append("<UNKOWN>")
#                 continue
        
#         tokens =  tok_list 

        tokens = parser(texts)
        tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
        tokens = [tok for tok in tokens if tok not in STOPLIST]
        tokens = [tok for tok in tokens if tok not in SYMBOLS]
        
        while "" in tokens:
            tokens.remove("")
        while " " in tokens:
            tokens.remove(" ")
        while "\n" in tokens:
            tokens.remove("\n")
        while "\n\n" in tokens:
            tokens.remove("\n\n")
                
        all_tokens = " ".join(tokens)
        token_dict[document.text] = str(all_tokens)
    
    # do a reverse lookup
    return_tokens = list(range(len_texts))
    for text, i in hash_table.items():
        try:
            return_tokens[i] = token_dict[text]
        except Exception as e:
            print("Tokenize Text error, setting to None")
            return_tokens[i] = "None"
            continue
    
    #print(return_tokens[:2])
    return return_tokens

               
               

IndentationError: unexpected indent (<ipython-input-29-8aaff6842c1a>, line 35)

In [107]:
# stopword that we dont want 
STOPLIST = set(stopwords.words('english')) # add more stopword
# symbols that we dont want, research add more later 
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”"]

# Lets create a custome tokenizer using spacy
def spacy_simple_tokenizer(texts):
    #tokens = parser(sentence)

    tokens = parser(texts)
    
#   print(dir(tokens))
#   print(set([w for w in tokens.ents]))
#   tokens = [tok.text.lower().strip() if tok.ent_type_ == "" else "<{}>".format(tok.ent_type_) for tok in tokens]
     
#     for ent in tokens.ents:
#         print(ent.text, ent.start_char, ent.end_char, ent.label_)
        
#     for tok in tokens:
#         if tok.ent_type_ != "":
#             print(tok.ent_type_)
        
    # For named entity we hve to replace them with their positional index
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    
       
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")
                
    all_tokens = " ".join(tokens)
   
    return str(all_tokens)

               
               

In [14]:
from sklearn.model_selection import train_test_split
#change above X if we r using tokenize and other nlp process
#Dropping the Nan values and info
df_train.dropna(inplace=True)
X = df_train['text']
y = df_train['label']

# do the similar think on headline (author ?? or source)
# name these to test , validation
X_train,  X_test,  y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=12345)

In [105]:
only_one_df = df_train['text'].iloc[0]
#only_one_df = "this is a cat, and a cat walked over to mountain and said what a lovely day is, and she was in China.\n Is India a nice place. "

In [109]:
#started at 11.54 -> took almost 1 hour 
for docs in df_train['text']:
    return_token = spacy_simple_tokenizer(docs)
    #return_token 



In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer  = TfidfVectorizer(tokenizer= spacy_simple_tokenizer, ngram_range=(1,1), max_df= 0.85, min_df= 2, max_features=3000)

In [ ]:
# started at 12.46
from timeit import default_timer as timer
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_validation_tfidf = tfidf_vectorizer.transform(X_test)
# STOP MY TIMER
elapsed_time = timer() - start # in seconds
print(elapsed_time)